
1) Необходимо собрать информацию о вакансиях на вводимую должность
(используем input или через аргументы)
с сайта hh.ru. Приложение должно анализировать несколько страниц сайта
(также вводим через input или аргументы). 

In [1]:
import numpy as np
import pandas as pd
#from pprint import pprint
import requests
#import re
from bs4 import BeautifulSoup as bs

In [2]:
# functions: choosing the job and getting the link to search vacancies by job's name
def get_link():
    link='https://spb.hh.ru/search/vacancy?area=2&st=searchVacancy&text=Программист'
    return link

# getting parsed html code
def parsing_page(link = get_link()):
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'})
    site_text = response.text
    parsed_page = bs(site_text, 'html.parser')

    return parsed_page
 
# getting an iterable object of 20 vacancies from the site's page
def searching_vacancy_block(parsed_site):
    vacancy_blocks = parsed_site.findAll('div', attrs={'data-qa': ['vacancy-serp__vacancy vacancy-serp__vacancy_premium', 'vacancy-serp__vacancy']})
    return vacancy_blocks

# getting a link to the next page of vacancies from the parsed html code
def searching_link(parsed_page):
    searching_link = parsed_page.find('a', attrs={'data-qa': 'pager-next'})
    link_2 = searching_link['href']
    new_link = f'https://spb.hh.ru{link_2}'
    return new_link

# extracting data on each vacancy from an object of 30 vacancies and restructuring it to pandas.dataframe
def extracting_vacancies(vacancy_block):
    vacancy_name = []
    vacancy_link = []
    vacancy_salary = []
    vacancy_resource =[]
    for vacancy in vacancy_block:
        # extracting vacancy's link
        link = vacancy.find('a')
        l = link['href']

        # storing it into a list variable
        vacancy_link.append(l)
        # extracting vacancy's name
        job_pos = vacancy.find('span', attrs={'class': 'g-user-content'})
        j = job_pos.get_text()

        # storing it into a list variable
        vacancy_name.append(j)
        # extracting the salary offered
        if vacancy.find('div', attrs={'class': 'vacancy-serp-item__compensation'}):
            job_sal = vacancy.find('div', attrs={'class': 'vacancy-serp-item__compensation'})
            s = job_sal.get_text()
             # storing it into a list variable
            vacancy_salary.append(s)
        else:
            vacancy_salary.append("По договоренности")
        
        vacancy_resource.append("HeadHunter")

        # lists of data into pandas.dataframe
    vacancy_df = pd.DataFrame({'Название вакансии': vacancy_name, 'Ссылка на вакансию': vacancy_link, 'Заработная плата': vacancy_salary, 'Источник вакансии': vacancy_resource},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата', 'Источник вакансии'])
    return vacancy_df



In [3]:
# forming an empty dataframe
vacancies_df_hh = pd.DataFrame({'Название вакансии': [], 'Ссылка на вакансию': [], 'Заработная плата': [], 'Источник вакансии':[]},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата', 'Источник вакансии'])

# calling function to get a parsed first page with vacancies
parsed_page = parsing_page()

# calling a function to get an object with 30 vacancies from the parsed page
vacancy_block = searching_vacancy_block(parsed_page)


In [4]:
len(vacancy_block)

20

In [5]:
# a cicle of collecting data from the first page, getting a link to the next page and collecting data from it too
while parsed_page.find('a', attrs={'data-qa': 'pager-next'}):
    vacancy_df = extracting_vacancies(vacancy_block)
    # appending formed dataframes from each page until there is no "next" page to parse
    vacancies_df_hh = vacancies_df_hh.append(vacancy_df)

    parsed_page = parsing_page(searching_link(parsed_page))
    vacancy_block = searching_vacancy_block(parsed_page)
    #limit to number of vacancies => limiting parsing process
    if len(vacancies_df_hh)>70:
        break

        

In [6]:
vacancies_df_hh.head()

,Название вакансии,Ссылка на вакансию,Заработная плата,Источник вакансии
0,Программист,https://spb.hh.ru/vacancy/34240290?query=%D0%9...,от 55 000 руб.,HeadHunter
1,JavaScript разработчик,https://spb.hh.ru/vacancy/34243260?query=%D0%9...,3 000-4 500 USD,HeadHunter
2,Программист 1 С (Ведущий программист 1 С),https://spb.hh.ru/vacancy/33370427?query=%D0%9...,60 000-150 000 руб.,HeadHunter
3,Программист 1C,https://spb.hh.ru/vacancy/33997193?query=%D0%9...,до 120 000 руб.,HeadHunter
4,"Программист С++, С#",https://spb.hh.ru/vacancy/33811538?query=%D0%9...,от 100 000 руб.,HeadHunter


In [7]:
vacancies_df_hh.to_csv('D:/AI/vacancies_hh.csv', index=False)